In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

# 1. Data loading
# 2. Data investigation and cleaning
# 3. Data visualization and engineering
# 4. Data preparation for modeling
# 5. Repeat 2-4 until get satisfied results

# file read

In [ ]:
#read from JSON
def load_df(csv_path='../input/train.csv', nrows=None):
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']

    df = pd.read_csv(csv_path, dtype={'fullVisitorId': 'str'}, nrows=nrows)

    for column in JSON_COLUMNS:
        df = df.join(pd.DataFrame(df.pop(column).apply(pd.io.json.loads).values.tolist(), index=df.index))

    return df

# data aggregation and merging

In [ ]:
gp_fullVisitorId_train = train.groupby(['fullVisitorId']).agg('sum')
gp_fullVisitorId_train['fullVisitorId'] = gp_fullVisitorId_train.index
gp_fullVisitorId_train['mean_hits_per_day'] = gp_fullVisitorId_train.groupby(['day'])['totals.hits'].transform('mean')
gp_fullVisitorId_train['mean_pageviews_per_day'] = gp_fullVisitorId_train.groupby(['day'])['totals.pageviews'].transform('mean')
gp_fullVisitorId_train['sum_hits_per_day'] = gp_fullVisitorId_train.groupby(['day'])['totals.hits'].transform('sum')
gp_fullVisitorId_train['sum_pageviews_per_day'] = gp_fullVisitorId_train.groupby(['day'])['totals.pageviews'].transform('sum')
gp_fullVisitorId_train = gp_fullVisitorId_train[['fullVisitorId', 'mean_hits_per_day', 'mean_pageviews_per_day', 'sum_hits_per_day', 'sum_pageviews_per_day']]
train = train.join(gp_fullVisitorId_train, on='fullVisitorId', how='inner', rsuffix='_')
train.drop(['fullVisitorId_'], axis=1, inplace=True)

In [ ]:
time_agg = train.groupby('date')['totals.transactionRevenue'].agg(['count', 'sum'])
year_agg = train.groupby('year')['totals.transactionRevenue'].agg(['sum'])
month_agg = train.groupby('month')['totals.transactionRevenue'].agg(['sum'])
day_agg = train.groupby('day')['totals.transactionRevenue'].agg(['sum'])
weekday_agg = train.groupby('weekday')['totals.transactionRevenue'].agg(['count','sum'])

In [ ]:
plt.figure(figsize=(20,7))
plt.ticklabel_format(axis='y', style='plain')
plt.ylabel('Sum transactionRevenue', fontsize=12)
plt.xlabel('Date', fontsize=12)
plt.scatter(time_agg.index.values, time_agg['sum'])
plt.show()

## one hot encoding and spiliting

In [ ]:
df_train_2 = pd.get_dummies(df_train, columns = ['wheezy-copper-turtle-magic'], prefix='wctm-')
df_test_2 = pd.get_dummies(df_test, columns = ['wheezy-copper-turtle-magic'], prefix='wctm-')
X_train_2, X_val_2, y_train_2, y_val_2 = train_test_split(df_train_2.drop(columns = ['id','target']),df_train_2['target'], test_size=0.15, random_state=2)

In [ ]:
categorical_features = ['device.isMobile','year', 'month', 'weekday', 'day']
train = pd.get_dummies(train, columns=categorical_features)
test = pd.get_dummies(test, columns=categorical_features)

## correlation and plot

In [ ]:
train_corr = train.iloc[:,:50].drop(["target", 'id'], axis=1).corr()
plt.figure(figsize=(10,10))
sns.heatmap(train_corr, vmin=-0.016, vmax=0.016, cmap="RdYlBu_r");

## feature selection - method
If there are multiple numeric columns and have different set of std, can have different levels in groups, to select useful columns, do modeling with mutitple (two) sets and observe the behavior, remove the weak ones. (reduce irrelative columns)
https://scikit-learn.org/stable/modules/feature_selection.html

In [ ]:
#method 1
sel = VarianceThreshold(threshold=1.5).fit(train2[cols])
train3 = sel.transform(train2[cols])
#method 2 PCA
StandardScaler().fit_transform(PCA(svd_solver='full',n_components='mle', whiten = 'True').fit_transform(data[cols]))
# Larsso, etc.

## feature selection - 2
PCA: reduce variables in order mainly to fix overfitting, if there is no overfitting, it will have worse result as information lost


## time conversion

In [ ]:
date_threshold = datetime.date(2019, 5, 29) #a single datetime.date point
date_threshold=pd.to_datetime(date_threshold) #convert to timestamp in order to compare with timestamp from pandas
rank_data['date'] = pd.to_datetime(rank_data['SubmissionDate']) #pandas convert str of time to timestamp
rank_data_0529 = rank_data[rank_data['date'] > date_threshold] #pandas column filter by time

def add_time_features(df):
    df['date'] = pd.to_datetime(df['date'], format='%Y%m%d', errors='ignore')
    df['year'] = df['date'].apply(lambda x: x.year)
    df['month'] = df['date'].apply(lambda x: x.month)
    df['day'] = df['date'].apply(lambda x: x.day)
    df['weekday'] = df['date'].apply(lambda x: x.weekday())
    
    return df

# create new dataframe


In [ ]:
meta = pd.DataFrame(data, columns = ['feature_name', 'col_name_1', 'col_name_2', 'col_name_3', 'col_name_4'])
meta.set_index('feature_name', inplace=True)

## data visualization 

In [ ]:
#multiple histogram overlap
plt.hist(rank_data_0529['Score'], bins=50, alpha=0.5, label='29', range=(0.9, 0.99)) 
plt.hist(rank_data_0528['Score'], bins=50, alpha=0.5, label='28', range=(0.9, 0.99)) 
plt.hist(rank_data_0527['Score'], bins=50, alpha=0.5, label='27', range=(0.9, 0.99))
plt.legend(loc='upper right')
plt.title('5/27-29')
plt.xlabel('auc')
plt.ylabel('Count')
plt.show()

In [ ]:
#overlay
plt.hist([data_arrays], bins=50, alpha=0.5, label=[names], range=(0.9, 0.99)) 
plt.legend(loc='upper right')
plt.title('5/27-29')
plt.xlabel('auc')
plt.ylabel('Count')
plt.show()

# print out results

In [ ]:
print('MSE: %.2f' % mse)


# prepare data for modeling

In [ ]:
# Get labels
Y_train = X_train['totals.transactionRevenue'].values
Y_val = X_val['totals.transactionRevenue'].values
X_train = X_train.drop(['totals.transactionRevenue'], axis=1)
X_val = X_val.drop(['totals.transactionRevenue'], axis=1)
test = test.drop(['totals.transactionRevenue'], axis=1)
# Log transform the labels
Y_train = np.log1p(Y_train)
Y_val = np.log1p(Y_val)

In [ ]:
# modeling

## stacking
Stacking (also called meta ensembling) is a model ensembling technique used to combine information from multiple predictive models to generate a new model.
For this reason, stacking is most effective when the base models are significantly different.
http://blog.kaggle.com/2016/12/27/a-kagglers-guide-to-model-stacking-in-practice/

In [ ]:
## LGBM
params = {
"objective" : "regression",
"metric" : "rmse", 
"num_leaves" : 500,
"min_child_samples" : 20,
"learning_rate" : 0.005,
"bagging_fraction" : 0.6,
"feature_fraction" : 0.7,
"bagging_frequency" : 1,
"bagging_seed" : 1,
"lambda_l1": 3,
'min_data_in_leaf': 70
}
lgb_train = lgb.Dataset(X_train, label=Y_train)
lgb_val = lgb.Dataset(X_val, label=Y_val)
model = lgb.train(params, lgb_train, 10000, valid_sets=[lgb_train, lgb_val], early_stopping_rounds=100, verbose_eval=100)

In [ ]:
# Make prediction on validation data.
val_predictions = model.predict(X_val, num_iteration=model.best_iteration) #validation
# Get min and max values of the predictions and labels.
min_val = max(max(val_predictions), max(Y_val))
max_val = min(min(val_predictions), min(Y_val))
# Create dataframe with validation predicitons and labels.
val_df = pd.DataFrame({"Label":Y_val})
val_df["Prediction"] = val_predictions
# Plot data
sns.set(style="darkgrid")
sns.jointplot(y="Label", x="Prediction", data=val_df, kind="reg", color="m", height=10)
plt.plot([min_val, max_val], [min_val, max_val], 'm--')
plt.show()
#feature importance
lgb.plot_importance(model, figsize=(15, 10))
plt.show()

val_predictions[val_predictions<0] = 0
mse = mean_squared_error(val_predictions, Y_val)
rmse = np.sqrt(mean_squared_error(val_predictions, Y_val))
print('Model validation metrics')
print('MSE: %.2f' % mse)
print('RMSE: %.2f' % rmse)
#prediction
predictions = model.predict(test, num_iteration=model.best_iteration)


# submission

In [ ]:
submission.to_csv("submission.csv", index=False)
